# widgets

> Widgets for building data systems.

- File
  - FolderSelector()
  - Folder_Button()
  
- System [psutil](https://pypi.org/project/psutil/)
  - CPU load, Percent, temperature
  - Memory
  - Disk 
    - Disk size
    - Disk space remaining

    


In [1]:
#| default_exp FileSystem

## Imports

### Module Imports

In [2]:
#| export
import csv
import datetime  as dt
import panel     as pn
import psutil    as ps
import sys
import threading as th
import time

from collections import namedtuple


### Hidden Imports

In [3]:
#| hide
from nbdev.showdoc  import *
from fastcore.test  import *
from fastcore.utils import *
import nbdev

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()

## Globals, Module wide

## Init calls for local testing.

In [5]:
pn.extension('floatpanel') 

## Functions

### CPU

#### def get_cpu_model():

In [6]:
#| export
def get_cpu_model( ) ->str: # `"model name"` string.
  '''
  Reads `"/proc/cpuinfo"` and returns the `"model name"` string.
  '''
  with open('/proc/cpuinfo') as f:
    lines = f.readlines()
  for i in lines:
    i = i.strip()              # Remove CRLF
    t = i.split('\t:')         # Split string by '\t:'
    if len(t) >1:              # 
      if t[0] == 'model name': # If we find "model name"
        rv = t[1]              #  then put in `rv` break out and return `rv`
        break
  return rv

In [7]:
# show_doc(get_cpu_model)

In [8]:
print(get_cpu_model() )

 Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz


#### def cpu_use_stats()

In [9]:
#| export
# return list has:
# scputimes(user=7815.26, nice=2.28, system=3641.48, idle=8743022.95, iowait=7.88, irq=0.0, softirq=669.33, steal=0.0, guest=0.0, guest_nice=0.0)
# wsl ww dt3 Execution time:
#   92.2 µs ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
def cpu_use_stats( # None
) -> tuple:  # A named tuple
  '''
  Returns CPU usage stats as a named tuple consisting of: 
  - totalsecs : Total CPU seconds the analysis is based on.
  - **all** : sum of: user+syst+iowait+softirq percent use.
  - **user** : user %
  - **system** : system %
  - **iowait** : iowait %
  - **softirq** : softirq %
  - **header** : A header string for str.
  - **str** : string variable with fully formatted values for each of above.
  '''
  v = ps.cpu_times()
  vtotal = 0.0
  for i in range(0,len(v)):
    vtotal += v[i]
  user    = 100*v.user/vtotal
  syst    = 100*v.system/vtotal
  iowait  = 100*v.iowait/vtotal
  softirq = 100*v.softirq/vtotal
  header  = 'Total time        All    User  System  IOwait softirq'
  str     = f'{vtotal/1e6:,f} sec  {user+syst+iowait+softirq:6.3f}% {user:6.3f}% { syst:6.3f}% {iowait:6.3f}% {softirq:6.3f}%'

  CPU_STATS = namedtuple('CPUStats', 'totalsecs all user system iowait softirq header str')
  cpu_stats = CPU_STATS(vtotal/1e6, user+syst+iowait+softirq, user, syst, iowait, softirq, header, str)
  return cpu_stats

In [10]:
#show_doc(cpu_use_stats)

Measure the average execution.

In [11]:
%%timeit -n 1000
cpu_stats = cpu_use_stats()

95.6 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Print the `header` and `str` elememts. These can generally be used as is.

cpu_stats.header,cpu_stats.str

## Widgets

###    def FolderSelector( parent, initial_dir='/', CallBack_func = None ):

In [12]:
#| export
def FolderSelector( parent,              # Parent widget
                   initial_dir='/',      # Dir to start in.
                   CallBack_func = None  # Function to call with selected folder.
                  ) ->object:            # The folder selector object
  """
  """
  def button_handler(event):
    floatpanel.status = 'closed'

    if event.obj.name   == 'Ok':
      parent.Selected_Folder = file_w._directory.value  
    elif event.obj.name == 'Cancel':
      parent.Selected_Folder = None

    if CallBack_func != None:
      CallBack_func( [parent.Selected_Folder, event]  )

  file_w          = pn.widgets.FileSelector(initial_dir,     root_directory ='/', refresh_period=500)
  FolderSel_b     = pn.widgets.Button(name='Ok',     button_type='primary')
  folder_cancel_b = pn.widgets.Button(name='Cancel', button_type='primary')
  folder_w        = pn.WidgetBox(file_w, pn.Row(FolderSel_b, folder_cancel_b) )
  floatpanel      = pn.layout.FloatPanel(folder_w, name='Basic FloatPanel', margin=20, contained=False, position='center')
  floatpanel.Selected_Folder = None
  FolderSel_b.on_click(    button_handler )
  folder_cancel_b.on_click( button_handler )
  parent.append( floatpanel )
  return floatpanel

Test & Debug: File_Selector( parent, initial_dir='/', CallBack_func = None ):

In [13]:
# Optional Function to be called once 'Ok' or 'Cancel' buttons are pressed.
def My_callBack( v ):
  debug_w.value = main_w.Selected_Folder

# Call back when selection button is pressed.  This will cause the
# FolderSelector to pop up on the screen.
def Button_commands( event ):
    w = FolderSelector( main_w, CallBack_func = My_callBack )

# Create a button widget to trigger the foldr selector popup
b_w     = pn.widgets.Button(name='Select Folder')

# Create a debug output text widget to show the selected folder.
debug_w = pn.widgets.StaticText(name='Selected Folder:', value='nothing selected yet.')

# Put the button and the debug text widget on a row together
main_w  = pn.Row( "Main", b_w, debug_w )

# Preload the main_w.Selected_Folder with None. This is to make sure the
# variable is defined.
main_w.Selected_Folder = None

# When the button is clicked, call Button_commands to pop up the folder selector
b_w.on_click( Button_commands )

# Show in the Jupyter output, or render with  panel 
main_w.servable()

Row
    [0] Markdown(str)
    [1] Button(name='Select Folder')
    [2] StaticText(name='Selected Folder:', value='nothing selected yet.')

In [16]:
print(main_w.Selected_Folder)

/


In [17]:
#| export
class Folder_Button:
  def __init__(self, 
               name='Select Folder',  # Folder Selector button name
               initial_dir = '/',     # Initial directory
               call_back = None,      # Call this function with selection when done.               
              ):
    """
    """
    self.Selected_Folder = None
    self.value           = None
    self.initial_dir   = initial_dir
    self.call_back     = call_back
    self.folder_button = pn.Row()                 # We need a container. Can't use button
    self.button        = pn.widgets.Button(name=name)    # 
    self.button.on_click( self.Button_commands )
    self.folder_button.append( self.button )

  def My_callBack(self, v ):
    self.value           = self.folder_button.Selected_Folder
    self.Selected_Folder = self.folder_button.Selected_Folder
    if self.call_back:
      self.call_back(self.value)
    
  def Button_commands(self, event ):
    self.w = FolderSelector( self.folder_button, 
                            initial_dir   = self.initial_dir,
                            CallBack_func = self.My_callBack )


In [18]:
#show_doc(FolderSelector)
#show_doc(Folder_Button)

In [19]:
def cb(v):
  print(f'cb = {v} ')

In [20]:
F = Folder_Button( name = "Folder",  call_back=cb )

In [21]:
F.folder_button

Row
    [0] Button(name='Folder')

In [23]:
F.value

'/boot'

### CPU

In [24]:
#| export
class CPU_WIDGET:
  '''
  '''
  Run             = False
  thread          = None
  thread_start_return_value  = None
  loop_count      = 0
  update_interval = 1
  utc_stamp       = dt.datetime.utcnow()
  utc_stamp_str   = utc_stamp.strftime("%Y-%m%d %H:%M:%S UTC")
  cpu_load_txt    = ps.getloadavg()
  cpu_count       = ps.cpu_count(logical=False)
  cpu_model_str   = get_cpu_model()
  core_usage      = list( range(0, cpu_count))

  def update_cpu_bars(self, value=None):
    '''
    {self.cpu_model_str:40s}
    '''
    self.utc_stamp        = dt.datetime.utcnow()
    self.utc_stamp_str    = self.utc_stamp.strftime("%Y-%m%d %H:%M:%S UTC")
    self.cpu_load_txt     = ps.getloadavg()
    self.cpu_load_w.value = f'{self.utc_stamp_str} Load Avgs:{self.cpu_load_txt}  {self.cpu_model_str:40s}'
    self.core_usage = cpu_state = ps.cpu_percent(percpu=True, interval=None)
    for cpu in range(self.cpu_count):
      self.cpu_bar[cpu].value = cpu_state[cpu]

  def system_status_thread(self, v ):
    '''
    '''
    self.Run = True
    while self.Run:
      self.update_cpu_bars()
      time.sleep( self.update_interval )
      self.loop_count +=  1
    self.cpu_load_w.value = f"Update Task Terminated normally at: {self.utc_stamp_str} "

  def start_thread( self ):
    self.thread = th.Thread( target = self.system_status_thread, args=[1111] )
    self.thread_start_return_value = self.thread.start()
    print( self.thread_start_return_value )
    
  def show_status(self):
    print('      Thread Name                         Run  Alive    Loops  Interval')
    print(f'{self.thread.name:40} {self.Run}   {self.thread.is_alive()} {self.loop_count:8d}   {self.update_interval:3.1f} Sec')
      
  def __init__(self):
    '''
    '''
    self.cpu_count = ps.cpu_count(logical=False)
    self.cpu_bar   = list(range(0, self.cpu_count))
    for cpu in range(self.cpu_count):
      self.cpu_bar[cpu] = pn.indicators.LinearGauge(name='',
                              value           =30, 
                              colors          =[(0.5, 'lightgreen'), (0.75,'yellow'), (1.0,'red')],
                              needle_color    ='', 
                              show_boundaries =False, 
                              default_color   ='lightgreen',
                              bounds          =(0, 100),
                              width           =50,
                              height          =200,
                              format          ='',
                              title_size      ='0pt',
                              tick_size       ='0pt',
                              margin          =(5,10),
                             )

    self.cpu_load_w = pn.widgets.StaticText(name='', value='Not set')
    self.row_w = pn.Row(f"**CPU<br>Core<br>% usage<br>{self.cpu_count} Cores**", name = 'xxxx', styles={'background': 'lightgray'})
    for cpu in range(0,self.cpu_count):
      self.row_w.append( self.cpu_bar[cpu]  )
    self.update_cpu_bars()
    self.cpu_status_widget = pn.Column(self.cpu_load_w, self.row_w)
    self.start_thread()
            

In [25]:
# show_doc( CPU_WIDGET )

In [26]:
cw = CPU_WIDGET()

None


In [54]:
cw.cpu_status_widget

Column
    [0] StaticText(value='2024-0110 04:14:38 UTC Lo...)
    [1] Row(name='xxxx', styles={'background': '...})
        [0] Markdown(str)
        [1] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [2] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [3] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [4] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [5] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [6] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [7] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [8] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)
        [9] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=2.7, width=50)
        [10] LinearGauge(colors=[(0.5, 'lightgreen'), ...], default_color='lightgreen', height=200, tick_size='0pt', title_size='0pt', value=0.0, width=50)

In [52]:
cw.show_status()


      Thread Name                         Run  Alive    Loops  Interval
Thread-5 (system_status_thread)          True   True      109   1.0 Sec


In [ ]:
cw.Run = False

#### Disks

In [36]:
disk_parts = ps.disk_partitions()
disk_parts

[sdiskpart(device='/dev/sde', mountpoint='/etc/resolv.conf', fstype='ext4', opts='rw,relatime,discard,errors=remount-ro,data=ordered', maxfile=255, maxpath=4096),
 sdiskpart(device='/dev/sde', mountpoint='/etc/hostname', fstype='ext4', opts='rw,relatime,discard,errors=remount-ro,data=ordered', maxfile=255, maxpath=4096),
 sdiskpart(device='/dev/sde', mountpoint='/etc/hosts', fstype='ext4', opts='rw,relatime,discard,errors=remount-ro,data=ordered', maxfile=255, maxpath=4096),
 sdiskpart(device='/dev/sdc', mountpoint='/home/jovyan/work', fstype='ext4', opts='rw,relatime,discard,errors=remount-ro,data=ordered', maxfile=255, maxpath=4096)]

In [37]:
for part in disk_parts:
  print(f'{part.device=} {part.mountpoint} {part.fstype} {part.opts}')

part.device='/dev/sde' /etc/resolv.conf ext4 rw,relatime,discard,errors=remount-ro,data=ordered
part.device='/dev/sde' /etc/hostname ext4 rw,relatime,discard,errors=remount-ro,data=ordered
part.device='/dev/sde' /etc/hosts ext4 rw,relatime,discard,errors=remount-ro,data=ordered
part.device='/dev/sdc' /home/jovyan/work ext4 rw,relatime,discard,errors=remount-ro,data=ordered


In [38]:
ps.disk_usage('/')

sdiskusage(total=269490393088, used=55620259840, free=200109461504, percent=21.7)

In [42]:
ps.disk_usage('/mnt/')

sdiskusage(total=269490393088, used=55620259840, free=200109461504, percent=21.7)

#### RAM
returned named tuple:

```
svmem(total=33598832640, available=30488162304, percent=9.3,
    used=2635255808, free=27074908160, active=4016775168,   
    inactive=1948282880, buffers=351911936, cached=3536756736,    
    shared=2940928, slab=387448832)
```


In [43]:
sys_m = ps.virtual_memory()
print('   Total    User    Free        Active   Buffers    Cached')
print(
f'{sys_m.available/1e9:6.2f}gb\
  {sys_m.used/1e9:6.2f}gb\
  {sys_m.free/1e9:6.2f}gb\
  {sys_m.active/1e9:6.2f}gb\
  {sys_m.buffers/1e9:6.2f}gb\
  {sys_m.cached/1e9:6.2f}gb'
)

   Total    User    Free        Active   Buffers    Cached
 30.42gb    2.62gb   14.77gb    5.28gb    1.31gb   14.89gb
